# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 10:36AM,249241,15,8686722,"Brookfield Pharmaceuticals, LLC",Released
1,Oct 24 2022 10:36AM,249241,15,8686735,"Brookfield Pharmaceuticals, LLC",Released
2,Oct 24 2022 10:36AM,249241,15,8686736,"Brookfield Pharmaceuticals, LLC",Released
3,Oct 24 2022 10:36AM,249241,15,8686737,"Brookfield Pharmaceuticals, LLC",Released
4,Oct 24 2022 10:36AM,249241,15,8686738,"Brookfield Pharmaceuticals, LLC",Released
5,Oct 24 2022 10:36AM,249241,15,8686744,"Brookfield Pharmaceuticals, LLC",Released
6,Oct 24 2022 10:36AM,249241,15,8686746,"Brookfield Pharmaceuticals, LLC",Released
7,Oct 24 2022 10:36AM,249241,15,8686747,"Brookfield Pharmaceuticals, LLC",Released
8,Oct 24 2022 10:36AM,249241,15,8686748,"Brookfield Pharmaceuticals, LLC",Released
9,Oct 24 2022 10:36AM,249241,15,8686749,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,249235,Released,1
25,249236,Released,2
26,249237,Released,25
27,249241,Released,27
28,249242,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249235,NaN,NaN,1.0
249236,NaN,NaN,2.0
249237,NaN,NaN,25.0
249241,NaN,NaN,27.0
249242,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249205,0.0,1.0,2.0
249209,1.0,15.0,3.0
249211,0.0,9.0,16.0
249212,0.0,8.0,2.0
249216,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249205,0,1,2
249209,1,15,3
249211,0,9,16
249212,0,8,2
249216,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249205,0,1,2
1,249209,1,15,3
2,249211,0,9,16
3,249212,0,8,2
4,249216,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249205,,1,2
1,249209,1,15,3
2,249211,,9,16
3,249212,,8,2
4,249216,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC"
27,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc."
28,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation
53,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation
55,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation
56,Oct 24 2022 10:09AM,249233,10,Bio-PRF
70,Oct 24 2022 10:06AM,249234,21,"NBTY Global, Inc."
71,Oct 24 2022 10:05AM,249230,10,Eywa Pharma Inc.
76,Oct 24 2022 10:04AM,249232,19,"Emersa Waterbox, LLC"
77,Oct 24 2022 10:03AM,249231,16,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",,,27
1,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",,,1
2,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation,,,25
3,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,,,2
4,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation,,,1
5,Oct 24 2022 10:09AM,249233,10,Bio-PRF,,,14
6,Oct 24 2022 10:06AM,249234,21,"NBTY Global, Inc.",,,1
7,Oct 24 2022 10:05AM,249230,10,Eywa Pharma Inc.,,,5
8,Oct 24 2022 10:04AM,249232,19,"Emersa Waterbox, LLC",,,1
9,Oct 24 2022 10:03AM,249231,16,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",27,,
1,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",1,,
2,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation,25,,
3,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,2,,
4,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation,1,,
5,Oct 24 2022 10:09AM,249233,10,Bio-PRF,14,,
6,Oct 24 2022 10:06AM,249234,21,"NBTY Global, Inc.",1,,
7,Oct 24 2022 10:05AM,249230,10,Eywa Pharma Inc.,5,,
8,Oct 24 2022 10:04AM,249232,19,"Emersa Waterbox, LLC",1,,
9,Oct 24 2022 10:03AM,249231,16,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",27,,
1,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",1,,
2,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation,25,,
3,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,2,,
4,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",27.0,NaN,NaN
1,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",1.0,NaN,NaN
2,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation,25.0,NaN,NaN
3,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,2.0,NaN,NaN
4,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 24 2022 10:36AM,249241,15,"Brookfield Pharmaceuticals, LLC",27.0,0.0,0.0
1,Oct 24 2022 10:31AM,249242,20,"Exact-Rx, Inc.",1.0,0.0,0.0
2,Oct 24 2022 10:25AM,249237,10,ISDIN Corporation,25.0,0.0,0.0
3,Oct 24 2022 10:13AM,249236,10,ISDIN Corporation,2.0,0.0,0.0
4,Oct 24 2022 10:09AM,249235,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4486018,370.0,41.0,1.0
15,249241,27.0,0.0,0.0
16,249231,1.0,0.0,0.0
19,747663,8.0,1.0,0.0
20,249242,1.0,0.0,0.0
21,249234,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4486018,370.0,41.0,1.0
1,15,249241,27.0,0.0,0.0
2,16,249231,1.0,0.0,0.0
3,19,747663,8.0,1.0,0.0
4,20,249242,1.0,0.0,0.0
5,21,249234,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,370.0,41.0,1.0
1,15,27.0,0.0,0.0
2,16,1.0,0.0,0.0
3,19,8.0,1.0,0.0
4,20,1.0,0.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,370.0
1,15,Released,27.0
2,16,Released,1.0
3,19,Released,8.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,41.0,0.0,0.0,1.0,0.0,0.0
Released,370.0,27.0,1.0,8.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,41.0,0.0,0.0,1.0,0.0,0.0
2,Released,370.0,27.0,1.0,8.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,41.0,0.0,0.0,1.0,0.0,0.0
2,Released,370.0,27.0,1.0,8.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()